In [1]:
import pandas as pd
import numpy as np
import ast
import string

movies = pd.read_csv("movie_data.csv")

movies['genres'] = pd.Series(ast.literal_eval(genres) for genres in movies['genres'])

movies.head(5)

,id,title,release_date,box_office_revenue,runtime,genres,summary
0,0,Ghosts of Mars,2001-08-24,14010832.0,98.0,"[Space western, Horror, Supernatural, Thriller...","Set in the second half of the 22nd century, th..."
1,1,White Of The Eye,1987,NaN,110.0,"[Erotic thriller, Psychological thriller, Thri...",A series of murders of rich young women throug...
2,2,A Woman in Flames,1983,NaN,106.0,[Drama],"Eva, an upper class housewife, becomes frustra..."
3,3,The Sorcerer's Apprentice,2002,NaN,86.0,"[Adventure, Fantasy, World cinema, Family Film]","Every hundred years, the evil Morgana returns..."
4,4,Little city,1997-04-04,NaN,93.0,"[Romance Film, Ensemble Film, Comedy-drama, Co...","Adam, a San Francisco-based artist who works a..."


## Here was my second attempt
Yet again we keep getting memory errors....

What words are characteristic of the movie summaries in those genres? Specifically:

- Drama
- Comedy
- Action/Adventure
- Romance film
- Thriller

In [2]:
unique_genres = ["Drama", "Comedy", "Action/Adventure", "Romance film", "Thriller"]

def tokenizer(blob):
    translator = str.maketrans({key: None for key in string.punctuation})
    
    tokenized = blob.lower().replace("-"," ").translate(translator).split()
    return tokenized

In [3]:
movie_dict = {}
for genre in unique_genres:
    for title, genres, summary in zip(movies['title'], movies['genres'], movies['summary']):
        if genre in genres:
            movie_dict[(title, genre)] = tokenizer(summary)
            
word_index = {word: n for summary in movie_dict.values() for n, word in enumerate(summary)}
word_index;

In [4]:
# movie_vecs = {(value[0], value[1]): np.zeros(len(word_index)) for value in movie_dict.keys()}
# movie_vecs
movie_vecs = {}
for title, genre in movie_dict.keys():
    movie_vecs[(title, genre)] = np.zeros(len(word_index))

MemoryError: 

## The code below is our first attmempt

In [2]:
#all_genres stores all the genres split out of the lists and stacks horizontally
#This makes is easy to construct our dataframe later
all_genres = np.hstack(movies.genres)

#Each feature is stacked so that a value repeats for as many genres as were in the original genre list
all_titles = np.hstack([[title]*len(genre) for title, genre in movies[['title', 'genres']].values])
all_release_dates = np.hstack([[release]*len(genre) for release, genre in movies[['release_date', 'genres']].values])
all_revenues = np.hstack([[revenue]*len(genre) for revenue, genre in movies[['box_office_revenue', 'genres']].values])
all_runtimes = np.hstack([[runtime]*len(genre) for runtime, genre in movies[['runtime', 'genres']].values])
all_summaries = np.hstack([[summary]*len(genre) for summary, genre in movies[['summary', 'genres']].values])

In [2]:
movies_split = pd.DataFrame({'genres': np.hstack(movies.genres), 
                             'titles':np.hstack([[title]*len(genre) for title, genre in movies[['title', 'genres']].values]), 
                             'release_date':np.hstack([[release]*len(genre) for release, genre in movies[['release_date', 'genres']].values]),
                             'box_office_revenue':np.hstack([[revenue]*len(genre) for revenue, genre in movies[['box_office_revenue', 'genres']].values]), 
                             'runtime':np.hstack([[runtime]*len(genre) for runtime, genre in movies[['runtime', 'genres']].values]), 
                             'summary':np.hstack([[summary]*len(genre) for summary, genre in movies[['summary', 'genres']].values])})

In [3]:
movies_split = pd.DataFrame({'genres':all_genres, 'titles':all_titles, 'release_date':all_release_dates,
                     'box_office_revenue':all_revenues, 'runtime':all_runtimes, 'summary':all_summaries})

#Let's check to see if this yields what we expect...
movies_split.head(10)

,box_office_revenue,genres,release_date,runtime,summary,titles
0,14010832.0,Space western,2001-08-24,98.0,"Set in the second half of the 22nd century, th...",Ghosts of Mars
1,14010832.0,Horror,2001-08-24,98.0,"Set in the second half of the 22nd century, th...",Ghosts of Mars
2,14010832.0,Supernatural,2001-08-24,98.0,"Set in the second half of the 22nd century, th...",Ghosts of Mars
3,14010832.0,Thriller,2001-08-24,98.0,"Set in the second half of the 22nd century, th...",Ghosts of Mars
4,14010832.0,Science Fiction,2001-08-24,98.0,"Set in the second half of the 22nd century, th...",Ghosts of Mars
5,14010832.0,Action,2001-08-24,98.0,"Set in the second half of the 22nd century, th...",Ghosts of Mars
6,14010832.0,Adventure,2001-08-24,98.0,"Set in the second half of the 22nd century, th...",Ghosts of Mars
7,NaN,Erotic thriller,1987,110.0,A series of murders of rich young women throug...,White Of The Eye
8,NaN,Psychological thriller,1987,110.0,A series of murders of rich young women throug...,White Of The Eye
9,NaN,Thriller,1987,110.0,A series of murders of rich young women throug...,White Of The Eye


In [3]:
# drama = movies_split[movies_split['genres'] == 'Drama']
# comedy = movies_split[movies_split['genres'] == 'Comedy']
# action_adv = movies_split[movies_split['genres'] == 'Action/Adventure']
# romance = movies_split[movies_split['genres'] == 'Romance Film']
# thriller = movies_split[movies_split['genres'] == 'Thriller']
# romance.head(10)

top = movies_split[(movies_split['genres'] == 'Drama') | (movies_split['genres'] == 'Comedy') |
                  (movies_split['genres'] == 'Action/Adventure') | (movies_split['genres'] == 'Romance Film') |
                  (movies_split['genres'] == 'Thriller')]
top.head(5)

,box_office_revenue,genres,release_date,runtime,summary,titles
3,14010832.0,Thriller,2001-08-24,98.0,"Set in the second half of the 22nd century, th...",Ghosts of Mars
9,NaN,Thriller,1987,110.0,A series of murders of rich young women throug...,White Of The Eye
10,NaN,Drama,1983,106.0,"Eva, an upper class housewife, becomes frustra...",A Woman in Flames
15,NaN,Romance Film,1997-04-04,93.0,"Adam, a San Francisco-based artist who works a...",Little city
18,NaN,Comedy,1997-04-04,93.0,"Adam, a San Francisco-based artist who works a...",Little city


In [5]:
sample = romance['summary'][15]
sample

NameError: name 'romance' is not defined

In [4]:
def tokenizer(blob):
    translator = str.maketrans({key: None for key in string.punctuation})
    
    tokenized = blob.lower().replace("-"," ").translate(translator).split()
    return tokenized

# tokens = tokenizer(sample)

In [5]:
summary_dict = {}
vocab = {}

for title, summary in zip(top['titles'], top['summary']):
    summary_dict[title] = tokenizer(summary)
    for token in tokenizer(summary):
        vocab[token] = vocab.get(token, 0) + 1

word_index = {}
for idx, word in enumerate(sorted(vocab)): 
        word_index[word] = idx

In [7]:
movie_vec = {title: np.zeros(len(word_index)) for title in top['titles']}

for title, summary in summary_dict.items():
    for word in summary:
        movie_vec[title][word_index[word]] += 1
movie_vec

MemoryError: 